In [26]:
#Step 1: Read CSV file into a dataframe

import pandas as pd

#Read in the files
df = pd.read_csv('Posts.csv')
df_users = pd.read_csv('Users.csv')


In [27]:
#Drop any posts that aren't questions or answers
df = df.drop(df[(df.PostTypeId > 2)].index)

In [28]:
df = df[['Id', 'PostTypeId', 'AcceptedAnswerId', 'ParentId', 'OwnerUserId', 'OwnerDisplayName']].copy()

df.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,OwnerUserId,OwnerDisplayName
0,1,1,173.0,NaN,15.0,NaN
1,2,1,9.0,NaN,11.0,NaN
2,3,2,NaN,1.0,30.0,NaN
3,4,2,NaN,1.0,28.0,NaN
4,5,1,15.0,NaN,33.0,NaN


In [29]:
#Fill NaN values for AcceptedAnswerId with 0
df['AcceptedAnswerId']=df['AcceptedAnswerId'].fillna(0.0)

#Step 2: Group the dataframe rows by PostTypeId to seperate questions and answers, and create a list 
#        of acceptable answer Ids

#Group Question and Answer posts and aggregate the accepted answer Ids into lists 
#       - Answer posts  (2.0) should only have 0 values
gf = df.groupby('PostTypeId').agg({'AcceptedAnswerId': lambda x: list(x)})

#Step 3: Store AcceptedAnswerIds in a list where PostTypeId equals 1.0
# This represents the Ids for the answers that have been accepted by the user who posted the question
# 0.0 represents the question posts that had no accepted answer

#Move the acceptable answer ids list from a dataframe column into a list
answer_ids = [answers for answers in gf.at[1, 'AcceptedAnswerId']]

#Sort the list in ascending order
answer_ids.sort()

In [30]:
#If it's an answer, assign the Id to ParentId
df.loc[df['PostTypeId'] == 2, 'FinalId'] = df['ParentId']
#If it's an accepted answer, assign the Id as Id
df.loc[df.Id.isin(answer_ids), 'FinalId'] = df['Id']
#If it's a question, assign the Id as Id
df.loc[df['PostTypeId'] == 1, 'FinalId'] = df['Id']


df.head(25)

,Id,PostTypeId,AcceptedAnswerId,ParentId,OwnerUserId,OwnerDisplayName,FinalId
0,1,1,173.0,NaN,15.0,NaN,1.0
1,2,1,9.0,NaN,11.0,NaN,2.0
2,3,2,0.0,1.0,30.0,NaN,1.0
3,4,2,0.0,1.0,28.0,NaN,1.0
4,5,1,15.0,NaN,33.0,NaN,5.0
5,6,1,16.0,NaN,28.0,NaN,6.0
6,7,2,0.0,2.0,12.0,NaN,2.0
7,8,2,0.0,2.0,7.0,NaN,2.0
8,9,2,0.0,2.0,28.0,NaN,9.0
9,10,1,14.0,NaN,32.0,NaN,10.0


In [31]:
df = df.drop(columns=['Id', 'PostTypeId','AcceptedAnswerId','ParentId'])

df.head()

,OwnerUserId,OwnerDisplayName,FinalId
0,15.0,NaN,1.0
1,11.0,NaN,2.0
2,30.0,NaN,1.0
3,28.0,NaN,1.0
4,33.0,NaN,5.0


In [32]:
df = df.rename(columns={"FinalId": "Id"})

df.head()

,OwnerUserId,OwnerDisplayName,Id
0,15.0,NaN,1.0
1,11.0,NaN,2.0
2,30.0,NaN,1.0
3,28.0,NaN,1.0
4,33.0,NaN,5.0


In [33]:
# Step 5: Storing the results in a new csv file

df.to_csv('gender-part1.csv')